## Тестовое задание от Maximum

In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from sqlalchemy import (create_engine, text, Table, Column, Integer, MetaData, REAL, TEXT, TIMESTAMP)




load_dotenv()

data_path = 'C:/Dev/MAXIMUM/max/ТЗ_Стажировка дата-аналитика_ Maximum Education.xlsx'

### Создание БД

In [2]:
user = os.getenv('user', '')
password = os.getenv('password', '')
host = os.getenv('host', '')
bd = os.getenv('bd', '')

engine = create_engine('postgresql://{}:{}@{}/{}'.format(user, password, host, bd))
metadata = MetaData()


managers = Table(
    'Managers', metadata,
    Column('id Manager', TEXT, primary_key=True),
    Column('BE_lvl1', TEXT),
    Column('BE_lvl2', TEXT),
    Column('BE_lvl3', TEXT)
)

agreements = Table(
    'Agreements', metadata,
    Column('id Agreement', TEXT, primary_key=True),
    Column('Amount', REAL),
    Column('id Manager', TEXT),
    Column('id Сounterparties', TEXT),
    Column('Season sale', TEXT),
    Column('Season training', TEXT),
    Column('Segment',  Integer)
)

payments = Table(
    'Payments', metadata,
    Column('id Payment', TEXT, primary_key=True),
    Column('id Agreement', TEXT),
    Column('Payment number',  Integer),
    Column('Scheduled payment date', TIMESTAMP),
    Column('Actual payment date', TIMESTAMP),
    Column('Scheduled payment amount',  Integer),
    Column('Actual payment amount',  Integer)
)

metadata.create_all(engine)



### Заполнение таблиц данными

In [3]:
managers_df = pd.read_excel(data_path, "Managers")
agreements_df = pd.read_excel(data_path, "Agreements")
payments_df = pd.read_excel(data_path, "Payments")


managers_df.to_sql("Managers", engine, if_exists='append', index=False)
agreements_df.to_sql("Agreements", engine, if_exists='append', index=False)
payments_df.to_sql("Payments", engine, if_exists='append', index=False)

512

In [4]:
def get_connection_db(query: str,
                      con = engine.connect())-> None:
    """Подключение к бд и получение инфы."""
    display(pd.io.sql.read_sql(sql=text(query), con = con))

### Вопрос №1
- Вывести количество договоров, уникальных учеников, объем продаж, договоров на уникальных учеников и средний чек предпродаж сгруппированных по сезону продаж.
- Условия:
1. Есть платеж Payment number=1
2.  Дата фактического платежа не пустая
3. Сортировка по объему продажи по убыванию
4. Предпродажей являются договоры, у которых сезон обучение позже, чем сезон продажи. Например, если сезон продажи 2022-2023, а сезон обучения 2023-2024, то такой договор будет являться предпродажей

In [5]:
query = '''WITH tt AS (SELECT "id Agreement", "Actual payment amount" AS amount
			FROM "Payments"
			WHERE "Payment number" = 1 AND "Actual payment date" IS NOT NULL)
SELECT "Season sale" AS "Season_sale"
	   , count("id Agreement") AS "Agreements"
	   , count(DISTINCT "id Сounterparties") AS "Student" 
	   , sum(CASE WHEN amount IS NULL THEN 0 ELSE amount end) AS "Amount"
	   , CASE
		   WHEN count(DISTINCT "id Сounterparties") = 0 THEN 0
		   ELSE round(count("id Agreement")::decimal / count(DISTINCT "id Сounterparties"), 1) END AS "Agreements_Сounterparties"
	   , CASE 
	   		WHEN round(avg(amount) FILTER (WHERE split_part("Season training", '-', 1) = split_part("Season sale", '-', 2)), 1) IS NULL THEN 0
	   		ELSE round(avg(amount) FILTER (WHERE split_part("Season training", '-', 1) = split_part("Season sale", '-', 2)), 1)
	   END
	      AS "Average_check"
FROM tt LEFT JOIN "Agreements" USING ("id Agreement")
GROUP BY "Season sale"
HAVING "Season sale" IS NOT NULL
ORDER BY "Amount" DESC;'''


get_connection_db(query)


,Season_sale,Agreements,Student,Amount,Agreements_Сounterparties,Average_check
0,2023-2024,274,158,9235032,1.7,26128.6
1,2022-2023,424,259,6993670,1.6,29737.5
2,2024-2025,4,4,176500,1.0,0.0


### Вопрос 2
У каждого ученика вывести его год выпуска и количество договоров
- Условия:
1. Есть платеж Payment number=1
2.  Дата фактического платежа не пустая
3. Сотрировка по количеству договоров по возрастанию
4. Необходимо учесть, что ученик может купить договор сезона обучения 2022-2023 и сегмент 10 класса, значит, что его год выпуска 2024 и т.д. Лучше всего продумать такую логику, при которой код будет работать долгие годы и не придется дописывать в код дополнительные условия

In [6]:
query = '''WITH tt AS (SELECT "id Agreement"
			FROM "Payments"
			WHERE "Payment number" = 1 AND "Actual payment date" IS NOT NULL)
SELECT "id Сounterparties" AS "id_Student"
	   , max(split_part("Season training", '-', 2)::integer) + (11 - max("Segment")) AS "Year_of_issue"
	   , count("id Agreement") AS "Agreements"
FROM tt LEFT JOIN "Agreements" USING ("id Agreement")
GROUP BY "id Сounterparties"
HAVING "id Сounterparties" IS NOT NULL 
ORDER BY "Agreements";'''



get_connection_db(query)

,id_Student,Year_of_issue,Agreements
0,4b1053f3-9024-ee11-8381-ba5c733cbf1f,2028,1
1,283244f1-1624-ec11-b6e5-000d3ab8afe1,2024,1
2,28e831b1-6086-ec11-93b0-000d3a64aefe,2025,1
3,2d0774a2-2b2b-ed11-9db1-000d3ab86c5b,2023,1
4,30e6f4d2-a0ff-ed11-8f6e-000d3aba1673,2027,1
...,...,...,...
384,06001d4b-143f-ed11-9db0-000d3aba1673,2028,5
385,ed377f34-e108-ec11-b6e6-000d3adda560,2024,6
386,d5d1f9af-2f42-e911-a999-000d3ab20be2,2025,6
387,3a0da3b7-0402-e911-a985-000d3ab5ccb7,2025,7


### Вопрос 3
Вывести количество просрочных платежей сгруппированных по подразделению lvl1, уникальное количество менеджеров, у которых есть просроченный платеж, менеджеров всего в отделе и %менеджеров с просроченными платежами. Подразделение lvl1 преобразовать так, что если в названии содержится "Старшая школа", то отображалось "ОП Старшая школа", а если содержится "Средняя школа" или "Екатеринбург+НН+НСК", то "ОП Средняя школа"
- Условия:
1. Дата фактического платежа не пустая
2. Дата фактического платежа позже, чем дата запланированного платежа
3. Объем фактического платежа не отрицательный и не пустой

In [7]:
query = '''WITH tt AS (SELECT "id Manager" AS bad_m
			FROM "Payments" RIGHT JOIN "Agreements" USING ("id Agreement")
			WHERE "Actual payment date" IS NOT NULL 
			     AND "Actual payment date" > "Scheduled payment date"
			     AND "Actual payment amount" IS NOT NULL 
			     AND "Actual payment amount" > 0), 
t_m AS (SELECT "id Manager"
                , CASE
                	WHEN "BE_lvl1" LIKE '% Старшая школа%' OR "BE_lvl1" LIKE '% Старшая школа %' THEN 'ОП Старшая школа'
					WHEN "BE_lvl1" LIKE '% Средняя школа%' OR "BE_lvl1" LIKE '% Средняя школа %' OR "BE_lvl1" LIKE 'Екатеринбург+НН+НСК%' THEN 'ОП Средняя школа'
				  END AS "BE_lvl1"
		FROM "Managers")
SELECT "BE_lvl1"
       , count(bad_m) AS "Late_payments" 
       , count(DISTINCT bad_m) AS "Late_managers"
       , count(DISTINCT "id Manager") AS "Managers"
       , round(count(DISTINCT bad_m)*100.0/ count(DISTINCT "id Manager"), 1) AS "%Late_managers"
FROM t_m AS m LEFT JOIN tt AS t ON m."id Manager" = t.bad_m
WHERE "BE_lvl1" IS NOT NULL 
GROUP BY "BE_lvl1";'''


get_connection_db(query)

,BE_lvl1,Late_payments,Late_managers,Managers,%Late_managers
0,ОП Средняя школа,100,17,193,8.8
1,ОП Старшая школа,124,21,257,8.2


Комментарии:

1) Нарушение целостности данных* В таблице `Agreements` присутствуют идентификаторы менеджеров (`id Manager`), которые отсутствуют в таблице `Managers`. Это нарушает целостность данных, поскольку `id Manager` в таблице `Agreements` должен быть внешним ключом, ссылкающимся на записи в таблице `Managers`. Аналогично, в таблице `Payments` присутствуют идентификаторы договоров (`id Agreement`), которые отсутствуют в таблице `Agreements`, что также нарушает целостность данных, поскольку `id Agreement` в таблице `Payments` должен быть внешним ключом, ссылкающимся на записи в таблице `Agreements`.

2) Несоответствие в наименовании столбцов:В ER-диаграмме в таблице `Agreements` есть столбец с названием `id Student`, который везде называется `id Сounterparties`. Это может вызвать путаницу или ошибки при интерпретации данных. 